In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
from wikidataintegrator import wdi_core, wdi_login
import pandas as pd
import re
import json

%run 'Shared.ipynb'
wdsparql = init_wd_sparql()
wbsparql = init_wb_sparql()
wb_endpoint, wb_login = init_wb()

In order to know the shape of the data we want to retrieve from Wikidata, we will look up a dynamic list of properties that are identifiers that can match entities to other data sets.

There are thousands of exteneral identifiers in Wikidata, so we will limit our search tosome that are found on some fairly well knows place items.

https://w.wiki/72sh

In [6]:
# Set the query
query = '''
SELECT DISTINCT ?property ?propertyLabel
WHERE {
  VALUES ?item {
    wd:Q243   # Eiffel Tower
    wd:Q41225 # Big Ben
    wd:Q9188  # Empire State Building
    wd:Q37200 # Great Pyramid of Giza
    wd:Q934   # North Pole
    wd:Q1899  # Kyiv
  }
  ?item ?p ?statement .
  ?property wikibase:claim ?p .
  ?property a wikibase:Property ;
            wikibase:propertyType wikibase:ExternalId .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
'''

# Set the SPARQL query and request JSON response
wdsparql.setQuery(query)

# Execute the SPARQL query
results = wdsparql.query().convert()

# Extract the property data from the results
properties = []
for result in results['results']['bindings']:
    prop_id = result['property']['value'].split('/')[-1]
    prop_label = result['propertyLabel']['value']
    properties.append({'property_id': prop_id, 'property_label': prop_label})

# Convert the list of properties to a Pandas DataFrame
df_properties = pd.DataFrame(properties)

# Print the DataFrame
print(df_properties)

    property_id                       property_label
0         P1669  Cultural Objects Names Authority ID
1         P3108                              Yelp ID
2         P4272                    DPLA subject term
3         P4986                 Routard.com place ID
4         P9346           France24 topic ID (French)
..          ...                                  ...
164      P11137                      Reddit topic ID
165       P8119                                 HASC
166       P7302                      Digital Giza ID
167       P8406                  Grove Art Online ID
168      P10397                         SBN place ID

[169 rows x 2 columns]


We will go ahead and create all of these properties on our Wikibase instance, so that we can use them in the future...

In [7]:
# Create properties that don't exist locally                                
for index, row in df_properties.iterrows():
    property_id = row['property_id']
    property_label = row['property_label']
    map_name = 'wikidata/' + property_id
    source_pid = known_lookup("source")
    source_id_pid = known_lookup("source-id")

    # Check if the property already exists
    if known_lookup(map_name) is not None:
        print("Property already exists: " + property_label)
        continue

    # Create the property
    s = [
        wdi_core.WDString("wikidata", prop_nr=source_pid),
        wdi_core.WDString(property_id, prop_nr=source_id_pid)
        ]
    property = wdi_core.WDItemEngine(mediawiki_api_url=wb_endpoint, data=s)
    property.set_label("Wikidata: " + property_label)
    property.set_aliases([map_name])

    # Write and catch errors, as wikibase.cloud can be a bit slow in allowing us to lookup existing things sometimes
    try:
        property.write(login=wb_login,entity_type='property', property_datatype='string')
        known_add(map_name, property.wd_item_id)
    except Exception as e:
        if 'label-conflict' in str(e):
            known_add(map_name, re.search(r'\[\[Property:([^|]+)\|', str(e)).group(1))
            print("Property already exists: (caught from error)")
        else:
            print("Error creating property: " + str(e))
            exit()

The query below looks for all locations that are 2.3 km away from the Empire State Building.
Viewable online at https://w.wiki/72rQ

In [8]:
# Set the base query template
base_query = '''SELECT ?place ?placeLabel ?location {extra_selects}
WHERE
{{
  wd:Q9188 wdt:P625 ?loc .
  SERVICE wikibase:around {{
      ?place wdt:P625 ?location .
      bd:serviceParam wikibase:center ?loc .
      bd:serviceParam wikibase:radius "2.3" .
  }}
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en" . }}
  {optional_clauses}
}}
'''

# Get the list of properties from the DataFrame
properties = df_properties['property_id'].tolist()

# Generate the SELECT clause and OPTIONAL clauses
select_clause = ' '.join(['?{}'.format(re.sub(r'\W+', '', p)) for p in properties])
optionals = '\n'.join(['OPTIONAL {{ ?place wdt:{} ?{} }}.'.format(p, re.sub(r'\W+', '', p)) for p in properties])

# Build the complete query by inserting the clauses into the base template
query = base_query.format(extra_selects=select_clause, optional_clauses=optionals)

# Execute the SPARQL query
wdsparql.setQuery(query)
results = wdsparql.query().convert()

# Process the query results and convert to DataFrame
bindings = results['results']['bindings']
data = [{k: v['value'] for k, v in binding.items()} for binding in bindings]
df_results = pd.DataFrame(data)

# Print the DataFrame
print(df_results)

                                          place  \
0        http://www.wikidata.org/entity/Q389336   
1        http://www.wikidata.org/entity/Q431339   
2        http://www.wikidata.org/entity/Q692137   
3        http://www.wikidata.org/entity/Q939753   
4        http://www.wikidata.org/entity/Q964460   
...                                         ...   
3161  http://www.wikidata.org/entity/Q112957671   
3162  http://www.wikidata.org/entity/Q119221764   
3163  http://www.wikidata.org/entity/Q119221846   
3164  http://www.wikidata.org/entity/Q119221860   
3165  http://www.wikidata.org/entity/Q119221894   

                               location                              P3417  \
0              Point(-73.9849 40.74558)  American-Academy-of-Dramatic-Arts   
1           Point(-73.985972 40.745278)                                NaN   
2     Point(-73.974722222 40.736388888)                                NaN   
3            Point(-73.984165 40.74329)                                NaN 

Now let's slam this data into a Wikibase...

In [ ]:
# Iterate over the DataFrame rows and create items in Wikibase
for index, row in df_results.iterrows():
    # Extract the wikidata id from the place that looks like "http://www.wikidata.org/entity/Q33341"
    wikidata_id = row['place'].split('/')[-1]

    # And extract the location from the location that looks like "Point(40.748433 -73.985656)"
    # TODO actually handle precision..?
    lat, long = row['location'].split('(')[1].split(')')[0].split(' ')

## ----------------------------
## TODO got here, but perhaps we should just import "Wikidata ID" property, instead of "source" and "id"...?
## Depends if we want to desing having multiple items per entity into the model or no.. probably not..



    # Use the query service to see if the item already exists
    query = '''
    SELECT ?item WHERE {{
        ?item wdt:{} wd:{} .
    }}
    '''.format(wikidata_map_property, wikidata_id)
    wbsparql.setQuery(query)
    results = wbsparql.query().convert()
    if len(results['results']['bindings']) > 0:
        print('Local Item already exists: {}'.format(results['results']['bindings'][0]['item']['value']))
        continue

    # Create the item
    s = [
        wdi_core.WDGlobeCoordinate(latitude=lat, longitude=long, precision=0.0001, prop_nr=location_property),
        wdi_core.WDString(wikidata_map_property, wikidata_id)
    ]
    item = wdi_core.WDItemEngine(mediawiki_api_url=wb_endpoint, data=s)
    item.set_label(row['placeLabel'])
    # Add a statement for every value in the row
    for k, v in row.items():
        if k in ['place', 'placeLabel', 'location']:
            continue
        if v:
            item.statements.append(wdi_core.WDString(property_map[k], v))

    

    


KeyError: 'P2002'

: 

: 

: 

: 

: 

: 

: 

: 